In [1]:
# Primerament importem totes les llibreries necessaries per completar el projecte.
import os
import requests
import argparse
import csv
import pathlib 
import time
from datetime import datetime
from bs4 import BeautifulSoup
import json

In [2]:
# Carreguem el fitxer de configuració a la variable config:
with open("configuration.json", "r") as configFile:
    config = json.load(configFile)

In [11]:
# Funcions per obtindre la llista del indexs del Ibex 35 de la borsa de Barcelona

# Get Header
def getHeaders():   
    return config['header']


# Get table Page        
def getPageTable(UrlPage):        
    table = None
    try:
        # Data for post form    
        data = {'punto':'indice'}
        page = requests.post(UrlPage,timeout=10)
        soup = BeautifulSoup(page.text, "html.parser")
        table = soup.find('table', attrs={'id': 'ctl00_Contenido_tblAcciones'})
    except requests.exceptions.Timeout:
        print("Timeout a la càrrega de la web")
    except:        
        print("Error inesperat")
    return table        

# Get Firm page and wait x seconds
def getFirmTablePage(UrlFirmPage):    
    table = None    
    try:
        t0 = time.time()
        # Data for post form            
        data = {'punto':'indice'}    
        page = requests.get(UrlFirmPage,timeout=10)                    
        soup = BeautifulSoup(page.text, "html.parser")                
        table = soup.find('table', attrs={'id': 'ctl00_Contenido_tblCapital'})                           
        # Wait time depending of response
        response_delay = time.time() - t0     
        time.sleep(10 * response_delay)
    except requests.exceptions.Timeout:
        print("-->Timeout a la càrrega de dada de la companyia")                
    except:                
        print("-->Error inesperat: " +  UrlFirmPage)
    return table

# Get the list of firm interesting values
def getFirmValues(UrlBase,firmPage):
    Url = UrlBase+firmPage        
    values = []
    table = getFirmTablePage(Url)        
    if (table is not None):                
        nroFila=0
        rowNum=0
        for row in table.find_all("tr"):
            if rowNum==1:
                colNum=0                
                for cell in row.find_all('td'):
                    if colNum==1:
                        value=[cell.text,""]              
                        values.append(value)                                
                        break;
                    colNum=colNum+1
            rowNum=rowNum+1       
    return values

# Set List with column page names
def getColNames(UrlBase,UrlRef):
    colList = []
    table = getPageTable(UrlBase+UrlRef)
    for tr in table.find_all("tr"):      
        # Cells Header
        cellsHead = tr.findAll('th')                
        if ((cellsHead is not None) and (len(cellsHead) > 0)):
            for cellHead in cellsHead:   
                colList.append(cellHead.text.strip())
            colList.append('Capitalización')
    return colList

# Set List with table page values
def getIndexs(UrlBase,UrlRef,bTableHead):
    indexList = []
    table = getPageTable(UrlBase+UrlRef)    
    # Extract table values and links
    rowList = []
    for tr in table.find_all("tr"):        
        colList = []               
        valuesFirm = []
        # Cells Header
        cellsHead = tr.findAll('th')                
        if ((cellsHead is not None) and (len(cellsHead) > 0)):
            if (bTableHead):
                for cellHead in cellsHead:        
                    value = [cellHead.text.strip(),'']                  
                    colList.append(value)                                                   
                colList.append(['Capitalización',''])
        else:
            # Cells value
            cells = tr.findAll('td')                        
            for cell in cells:
                href = ''
                infComp = []
                a= cell.find('a')                  
                if (a is not None):                 
                    href = a.get('href')                                
                    # Append / if needed
                    if href[0] !='/':
                        href = '/' + href                     
                    valuesFirm = getFirmValues(UrlBase,href) 
                    # print firm
                    print(cell.text)
                value = [cell.text.strip(),href]                                  
                colList.append(value)             
                
        # Extend list with firm information 
        colList.extend(valuesFirm)        
        # Append to rowlist    
        rowList.append(colList)         
    return rowList 


def writecsv(table,filename):
    currentDir = os.path.abspath('')
    filePath = os.path.join(currentDir,filename)
    file = open(filePath,"a+")
    for row in table:
        for col in row:
            file.write(col[0]+";")
        file.write("\n")
    
# Extract all information of the Stock Exchange at any time
def SaveOneDayInfo(fileName,bTabHeader):
    indexList = getIndexs(config['URLs']['base'], config['URLs']['ref'], bTabHeader);       
    # Check end of Stock Exchange session
    bEndSession = False    
    if (indexList is not None):        
        posRow = 0
        if (bTabHeader): posRow = 1            
        HourSession = str(indexList[posRow][8][0])            
        bEndSession  = (HourSession.strip() == 'Cierre')
    #save file
    writecsv(indexList,fileName)            
    #return state
    return bEndSession        
        
# Save the Stock Exchange one day
def ScheduleDayFile():
    fileName = "ListaIndices" + time.strftime('%Y-%m-%d_%H-%M')+ ".csv"
    bEndSession = SaveOneDayInfo(fileName,True);
    while (not bEndSession):            
        print("Waiting 15 minutes")
        time.sleep(60*15)        
        bEndSession = SaveOneDayInfo(fileName,False);    
    print("End of session: file generated: " + fileName)
    
# Save the Stock Exchange one week
def ScheduleWeekFiles():    
    scheduler = BlockingScheduler()    
    scheduler.add_job(job_function, 'cron', day_of_week='mon-fri', hour=9, minute=0)    
    scheduler.start()    
    
    

In [13]:
# indicar la ruta
indexList = getIndexs(config['URLs']['base'], config['URLs']['ref'], True);
print(indexList)

ACCIONA
ACERINOX
ACS
AENA
ALMIRALL
AMADEUS
ARCELORMIT.
B.SANTANDER
BA.SABADELL
BANKINTER
BBVA
CAIXABANK
CELLNEX
CIE AUTOMOT.
ENAGAS
ENDESA
FERROVIAL
FLUIDRA
GRIFOLS CL.A
IAG
IBERDROLA
INDITEX
INDRA A
INM.COLONIAL
MAPFRE
MELIA HOTELS
MERLIN
NATURGY
PHARMA MAR
R.E.C.
REPSOL
SIEMENS GAME
SOLARIA
TELEFONICA
VISCOFAN
[[['Nombre', ''], ['Últ.', ''], ['% Dif.', ''], ['Máx.', ''], ['Mín.', ''], ['Volumen', ''], ['Efectivo (miles €)', ''], ['Fecha', ''], ['Hora', ''], ['Capitalización', '']], [['ACCIONA', '/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0125220311'], ['138,2000', ''], ['0,00', ''], ['139,9000', ''], ['137,9000', ''], ['18.687', ''], ['2.595,81', ''], ['08/04/2021', ''], ['12:29:43', ''], ['7.581.189', '']], [['ACERINOX', '/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0132105018'], ['11,2650', ''], ['0,27', ''], ['11,3200', ''], ['11,1750', ''], ['131.388', ''], ['1.477,75', ''], ['08/04/2021', ''], ['12:32:06', ''], ['3.039.586', '']], [['ACS', '/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES

In [5]:
writecsv(indexList, config["outputFile"])

In [8]:
ScheduleDayFile()
# Inspect tr - td -a href = Index site URL ref (append to base URL)

ACCIONA
ACERINOX
ACS
AENA
ALMIRALL
AMADEUS
ARCELORMIT.
B.SANTANDER
BA.SABADELL
BANKINTER
BBVA
CAIXABANK
CELLNEX
CIE AUTOMOT.
ENAGAS
ENDESA
FERROVIAL
FLUIDRA
GRIFOLS CL.A
IAG
IBERDROLA
INDITEX
INDRA A
INM.COLONIAL
MAPFRE
MELIA HOTELS
MERLIN
NATURGY
PHARMA MAR
R.E.C.
REPSOL
SIEMENS GAME
SOLARIA
TELEFONICA
VISCOFAN
Waiting 15 minutes
ACCIONA
ACERINOX
ACS
AENA
ALMIRALL
AMADEUS
ARCELORMIT.
B.SANTANDER
BA.SABADELL
BANKINTER
BBVA
CAIXABANK
CELLNEX
CIE AUTOMOT.
ENAGAS
ENDESA
FERROVIAL
FLUIDRA
GRIFOLS CL.A
IAG
IBERDROLA
INDITEX
INDRA A
INM.COLONIAL
MAPFRE
MELIA HOTELS
MERLIN
NATURGY
PHARMA MAR
R.E.C.
REPSOL
SIEMENS GAME
SOLARIA
TELEFONICA
VISCOFAN


IndexError: list index out of range

In [12]:
val = getColNames(config['URLs']['base'], config['URLs']['ref'])
print(val)

['Nombre', 'Últ.', '% Dif.', 'Máx.', 'Mín.', 'Volumen', 'Efectivo (miles €)', 'Fecha', 'Hora', 'Capitalización']
